In [1]:
# Build Baseline dataset from all valid trials from Baseline sessions in the AFIRM and BAA classifications
# File structure from root_dir:
# {root_dir}/
# ├── {classification}/
# │   ├── {subject}/
# │   │   ├── {session}/
# │   │   │   ├── {trial}.c3d
# │   │   │   └── ...
# │   │   └── ...
# │   └── ...
# └── ...

root_dir = 'data'
classifications = ['AFIRM', 'BAA']
session = 'Baseline'

from utils import scandir_regex
from rat_trial import RatTrial
import os
import numpy as np

file_pattern = f"{root_dir}/({'|'.join(classifications)})/.*/{session}/Static.*\\.c3d"
print(f"Searching for files with pattern: {file_pattern}")

baseline_files = scandir_regex(root_dir, file_pattern)
for file in baseline_files:
    print(f"Processing file: {file}")
    trial = RatTrial.from_c3d_file(file)
    # Validate
    if not trial.validate_trial():
        print(f"Invalid trial: {file}")
        continue
    trial.to_trc(file.replace('.c3d', '.trc'), 
             rotation = np.array([[1, 0, 0],
                                  [0, 0, 1],
                                  [0, -1, 0]])
             )
    trial.scale_opensim_model(
        unscaled_model_path='opensim/Bilateral/FinalBilateral.osim',
        marker_set_path='opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml',
        marker_file_name= os.path.basename(file).replace('.c3d', '.trc'),
        output_dir=os.path.dirname(file),
        scale_setup_path='opensim/Bilateral/generic_scale_setup_bilateral.xml'
    )


Searching for files with pattern: data/(AFIRM|BAA)/.*/Baseline/Static.*\.c3d
Processing file: data/BAA/BAA01/Baseline/Static01.c3d
[info] Processing subject Static01...
[info] Step 1: Loading generic model
[info] Updating Model file from 40000 to latest format...
[info] Loaded model FinalBilateral from file /home/hudson/Rats_Python/opensim/Bilateral/FinalBilateral.osim
[info] Loading marker set from '/home/hudson/Rats_Python/data/BAA/BAA01/Baseline//home/hudson/Rats_Python/opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml'.
[info] Updated markers in model FinalBilateral
[info] Step 2: Scaling generic model
[info] Loaded marker file /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01.trc (12 markers, 676 frames)
[info] Measurement 'spine'
[info] 	pair 0 (TAIL, SPL6): model = 0.0393357, experimental = 0.0352886
[info] 	overall scale factor = 0.897113
[info] Measurement 'pelvis_length'
[info] 	pair 0 (RASI, RHIP): model = 0.0299833, experimental = 0.0221034
[info] 	pair 1 (L

In [1]:
from rat_trial import RatTrial
import numpy as np
trial = RatTrial.from_c3d_file('data/BAA/BAA01/Baseline/Walk05.c3d')
# trial.to_trc('data/BAA/BAA01/Baseline/Walk05.trc', 
#              rotation = np.array([[1, 0, 0],
#                                   [0, 0, 1],
#                                   [0, -1, 0]])
#              )
# trial.run_opensim_ik('data/BAA/BAA01/Baseline/Static01_marker.osim', 
#                      trial.get_linked_file("trc"), 
#                      ik_setup_path='opensim/Bilateral/generic_Rat_Setup_IK.xml', 
#                      output_dir='data/BAA/BAA01/Baseline',
#                      start_time=trial.events[0].get_time(trial.points.rate),
#                      end_time=trial.events[-1].get_time(trial.points.rate)
#                      )

# Read .enf file to determine applied bodies
def parse_enf_file(file_path: str) -> dict:
    data = {}
    with open(file_path, 'r', errors='ignore') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                if key and value:
                    data[key.lower()] = value  # Ensure keys are lowercase for consistency
    return data

enf_data = parse_enf_file('data/BAA/BAA01/Baseline/Walk05.Trial.enf')
# Find keys that match the pattern 'fp%d' where %d is a digit and extract the digit and value
contacted_fp = {}
for key, value in enf_data.items():
    if key.startswith('fp') and key[2:].isdigit():
        contacted_fp[int(key[2:])] = 'foot_l' if str(value).lower() == 'left' else 'foot_r' if str(value).lower() == 'right' else value
print("Contacted force platforms found in .enf file:", contacted_fp)
# Create a list of applied bodies based on the contacted force platforms



trial.export_force_platforms(output_dir='data/BAA/BAA01/Baseline', 
                             applied_bodies= contacted_fp, 
                             rotation= np.array([[1, 0, 0],
                                                 [0, 0, 1],
                                                 [0, -1, 0]]) 
                            )



2025-06-24 03:21:23.518 | WARNING  | trial:export_force_platforms:875 - Force platform 1 position unit mm does not match output unit m. Converting positions.
2025-06-24 03:21:23.518 | WARNING  | trial:export_force_platforms:879 - Force platform 1 torque unit Nmm does not match output unit Nm. Converting torques.
2025-06-24 03:21:23.519 | WARNING  | trial:export_force_platforms:875 - Force platform 2 position unit mm does not match output unit m. Converting positions.
2025-06-24 03:21:23.519 | WARNING  | trial:export_force_platforms:879 - Force platform 2 torque unit Nmm does not match output unit Nm. Converting torques.
2025-06-24 03:21:23.520 | WARNING  | trial:export_force_platforms:875 - Force platform 3 position unit mm does not match output unit m. Converting positions.
2025-06-24 03:21:23.520 | WARNING  | trial:export_force_platforms:879 - Force platform 3 torque unit Nmm does not match output unit Nm. Converting torques.
2025-06-24 03:21:23.521 | WARNING  | trial:export_force_pl

Contacted force platforms found in .enf file: {3: 'foot_r', 4: 'foot_l'}
Writing 36 labels to MOT file: ['force1_vx', 'force1_vy', 'force1_vz', 'moment1_x', 'moment1_y', 'moment1_z', 'force1_px', 'force1_py', 'force1_pz', 'force2_vx', 'force2_vy', 'force2_vz', 'moment2_x', 'moment2_y', 'moment2_z', 'force2_px', 'force2_py', 'force2_pz', 'force3_vx', 'force3_vy', 'force3_vz', 'moment3_x', 'moment3_y', 'moment3_z', 'force3_px', 'force3_py', 'force3_pz', 'force4_vx', 'force4_vy', 'force4_vz', 'moment4_x', 'moment4_y', 'moment4_z', 'force4_px', 'force4_py', 'force4_pz']


In [ ]:
# Build pathology groups
classification = 'AFIRM'
NR = [
    "LGS09",
    "LGS10",
    "LGS13",
    "LGS14",
    "E17",
    "E20",
    "E21",
    "LGS11",
    "LGS12",
    "LGS15",
    "LGS16",
    "T05",
    "T06",
    "E16",
    "E18",
    "E19",
    "LGS08",
    "T08",
    "E08",
    "E12",
    "E13",
    "E14",
    "E15",
    "E26",
    "LGS05",
    "LGS06",
    "LGS07",
    "T07"
]
TEMR = [
    "T01",
    "T02",
    "T03",
    "T04",
    "T09",
    "T10",
    "T11",
    "T12",
    "G14_E23",
    "E09",
    "E10",
    "A04",
    "A05",
    "A06",
    "A07",
    "A08"
]
HH = [
    "G01",
    "G02",
    "G03",
    "G04",
    "G05",
    "G06",
    "G07",
    "G08",
    "LGSH01",
    "LGSH02",
    "LGSH03",
    "LGSH04",
    "G13",
    "G14",
    "G15",
    "G16",
    "G17",
    "G18"
]
TEMR_HH = [
    "H01",
    "H02",
    "H03",
    "H04",
    "H05",
    "H06",
    "H07",
    "H08",
    "G16_E24",
    "E02",
    "E06",
    "E11",
    "A09",
    "A10",
    "A11",
    "A12"
]
HS = [
    "S01",
    "S03",
    "S04",
    "S05",
    "S06",
    "S07",
    "S08",
    "S13",
    "S09",
    "S10",
    "S11",
    "S12",
    "S14",
    "S15",
    "S16",
    "S17"
]
TEMR_KG = [
    "K01",
    "K02",
    "K03",
    "K04",
    "K05",
    "K06",
    "K07",
    "K08",
    "G13_E22",
    "E01",
    "E03",
    "E04",
    "E05",
    "A01",
    "A02",
    "A03"
]

groups = {
    "NR": NR,
    "TEMR": TEMR,
    "HH": HH,
    "TEMR_HH": TEMR_HH,
    "HS": HS,
    "TEMR_KG": TEMR_KG
}

# Load kinematics and kinetics

# Spline to events
from scipy.interpolate import CubicSpline

# Group ( maybe with xarray )

# Plot